In [1]:
import glob
import time
import pandas as pd
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...


In [2]:

from tensorflow import keras
import pandas as pd
import os
import numpy as np

In [4]:

df=pd.read_csv('./final.csv',encoding='utf-8')

In [5]:
df.head()

,Unnamed: 0,Sentence,Label
0,0,""" or pg_sleep ( __TIME__ ) --",1
1,1,create user name identified by pass123 tempora...,1
2,2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,3,select * from users where id = '1' or @ @1 ...,1
4,4,"select * from users where id = 1 or 1#"" ( ...",1


In [7]:
del df['Unnamed: 0']

In [8]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25674 entries, 0 to 25673
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  25674 non-null  object
 1   Label     25674 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 401.3+ KB


In [16]:
values = df.values

In [17]:



# vectorization of data

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( min_df=2, max_df=0.7, max_features=4096, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(df['Sentence'].values.astype('U')).toarray()

In [18]:
posts.shape

(25674, 4096)

In [20]:
posts.shape=(25674,64,64,1)

In [21]:
posts = posts/np.max(posts)
X=posts
np.max(posts)

1.0

In [22]:
y=df['Label']

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [25]:
model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(64,64,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [26]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 256)        0

In [27]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=128)

Epoch 1/10
161/161 [==============================] - 95s 578ms/step - loss: 0.3193 - accuracy: 0.8590 - val_loss: 0.1993 - val_accuracy: 0.9281
Epoch 2/10
161/161 [==============================] - 87s 541ms/step - loss: 0.1725 - accuracy: 0.9393 - val_loss: 0.1534 - val_accuracy: 0.9451
Epoch 3/10
161/161 [==============================] - 86s 536ms/step - loss: 0.1424 - accuracy: 0.9488 - val_loss: 0.1419 - val_accuracy: 0.9482
Epoch 4/10
161/161 [==============================] - 86s 537ms/step - loss: 0.1248 - accuracy: 0.9573 - val_loss: 0.1299 - val_accuracy: 0.9548
Epoch 5/10
161/161 [==============================] - 88s 548ms/step - loss: 0.1163 - accuracy: 0.9603 - val_loss: 0.1288 - val_accuracy: 0.9554
Epoch 6/10
161/161 [==============================] - 87s 538ms/step - loss: 0.1139 - accuracy: 0.9611 - val_loss: 0.1192 - val_accuracy: 0.9599
Epoch 7/10
161/161 [==============================] - 87s 540ms/step - loss: 0.1091 - accuracy: 0.9623 - val_loss: 0.1173 - val_ac

In [28]:
from keras.models import load_model
import pickle

model.save('cnn.h5')
with open('vectorizer_cnn', 'wb') as fin:
    pickle.dump(vectorizer, fin)

In [3]:
import keras
from keras.models import load_model
import pickle

mymodel = load_model('cnn.h5')
myvectorizer = pickle.load(open("vectorizer_cnn", 'rb'))




def clean_data(input_val):

    input_val=input_val.replace('\n', '')
    input_val=input_val.replace('%20', ' ')
    input_val=input_val.replace('=', ' = ')
    input_val=input_val.replace('((', ' (( ')
    input_val=input_val.replace('))', ' )) ')
    input_val=input_val.replace('(', ' ( ')
    input_val=input_val.replace(')', ' ) ')
    input_val=input_val.replace('1 ', 'numeric')
    input_val=input_val.replace(' 1', 'numeric')
    input_val=input_val.replace("'1 ", "'numeric ")
    input_val=input_val.replace(" 1'", " numeric'")
    input_val=input_val.replace('1,', 'numeric,')
    input_val=input_val.replace(" 2 ", " numeric ")
    input_val=input_val.replace(' 3 ', ' numeric ')
    input_val=input_val.replace(' 3--', ' numeric--')
    input_val=input_val.replace(" 4 ", ' numeric ')
    input_val=input_val.replace(" 5 ", ' numeric ')
    input_val=input_val.replace(' 6 ', ' numeric ')
    input_val=input_val.replace(" 7 ", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace('1234', ' numeric ')
    input_val=input_val.replace("22", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace(" 200 ", ' numeric ')
    input_val=input_val.replace("23 ", ' numeric ')
    input_val=input_val.replace('"1', '"numeric')
    input_val=input_val.replace('1"', '"numeric')
    input_val=input_val.replace("7659", 'numeric')
    input_val=input_val.replace(" 37 ", ' numeric ')
    input_val=input_val.replace(" 45 ", ' numeric ')

    return input_val








def predict_sqli_attack():
    
    repeat=True
    
    beautify=''
    for i in range(20):
        beautify+= "="

    print(beautify) 
    input_val=input("Give me some data to work on : ")
    print(beautify)

    
    if input_val== '0':
        repeat=False
    
    

    input_val=clean_data(input_val)
    input_val=[input_val]



    input_val=myvectorizer.transform(input_val).toarray()

    result=mymodel.predict(input_val)


    print(beautify)
    
    
    if repeat == True:
        
        if result>0.5:
            print("ALERT :::: This can be SQL injection")


        elif result<=0.5:
            print("It seems to be safe")
            
        print(beautify)
            
        predict_sqli_attack()
            
    elif repeat == False:
        print( " Good Bye ")

 


 

In [4]:
predict_sqli_attack()

ValueError: in user code:

    File "c:\Users\ASUS\miniconda3\envs\datasci\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\ASUS\miniconda3\envs\datasci\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\ASUS\miniconda3\envs\datasci\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\ASUS\miniconda3\envs\datasci\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "c:\Users\ASUS\miniconda3\envs\datasci\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\ASUS\miniconda3\envs\datasci\lib\site-packages\keras\engine\input_spec.py", line 295